In [ ]:
%load_ext autoreload
%autoreload 2
%cd ..

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import pickle
from typing import Dict, Tuple, List
from collections import Counter

import matplotlib.pyplot as plt
from matplotlib.dates import date2num
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import scipy.signal
import scipy.fftpack

from util.paths import DATA_PATH
from util.datasets import SlidingWindowDataset, read_physionet_dataset, RespiratoryEventType
from util.filter import apply_butterworth_bandpass_filter, apply_butterworth_lowpass_filter
from util.mathutil import get_peaks, PeakType, cluster_1d, IntRange
from rule_based import detect_respiratory_events
from rule_based.detector import _detect_airflow_resp_events
from util.event_based_metrics import EventBasedConfusionMatrix, get_overlaps, get_n_detected_annotations

# Makes numpy raise errors instead of outputting warnings
np.seterr('raise')

# Some preparations to pretty-print tensors & ndarrays
np.set_printoptions(edgeitems=10)
np.core.arrayprint._line_width = 400

In [ ]:
dataset_names = ("tr03-0005", "tr03-0289", "tr03-0921", "tr04-1078", "tr07-0168")
print(f"Running detector over {len(dataset_names)} datasets..")
print()

confusion_matrix = EventBasedConfusionMatrix.empty()
n_annotated_events: int = 0
n_detected_events: int = 0
n_detected_annotations: int = 0
for dataset_name in tqdm(dataset_names):
    dataset_folder = DATA_PATH / "training" / dataset_name
    config = SlidingWindowDataset.Config(
        physionet_dataset_folder=dataset_folder,
        downsample_frequency_hz=5,
        time_window_size=pd.Timedelta("5 minutes")
    )
    sliding_window_dataset = SlidingWindowDataset(config=config, allow_caching=True)
    detected_respiratory_events = detect_respiratory_events(sliding_window_dataset.signals, sample_frequency_hz=sliding_window_dataset.config.downsample_frequency_hz, discard_wake_stages=False)

    n_annotated_events += len(sliding_window_dataset.respiratory_events)
    n_detected_events += len(detected_respiratory_events)

    cm_ = EventBasedConfusionMatrix(annotated_events=sliding_window_dataset.respiratory_events, detected_events=detected_respiratory_events)
    confusion_matrix += cm_
    o_ = get_n_detected_annotations(annotated_events=sliding_window_dataset.respiratory_events, detected_events=detected_respiratory_events)
    n_detected_annotations += o_

print()
print(f"Number of annotated respiratory events: {n_annotated_events}")
print(f"Number of detected respiratory events: {n_detected_events}")
print()
print(f"Number of detected annotations (overlaps): {n_detected_annotations}")
print(f" -> Annotation recall: {n_detected_annotations/n_annotated_events:.3f}")

In [ ]:
macro_scores = confusion_matrix.get_macro_scores()
print("Confusion-matrix based macro scores:")
print(f" -> {macro_scores}")

plt.figure(figsize=(7, 7))
confusion_matrix.plot()
